In [1]:
library(tidyverse)
memory.limit(size = 70000) 

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.4.0      v purrr   0.3.4 
v tibble  3.1.6      v dplyr   1.0.10
v tidyr   1.2.0      v stringr 1.4.1 
v readr   2.1.2      v forcats 0.5.1 
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


[1] 70000

In [2]:
sales_Data <- read.csv('competitive-data-science-predict-future-sales/sales_train.csv')
testData <- read.csv('competitive-data-science-predict-future-sales/test.csv')
itemsData <- read.csv('competitive-data-science-predict-future-sales/items.csv')
shopsData <- read.csv('competitive-data-science-predict-future-sales/shops.csv')
itemsCatData <- read.csv('competitive-data-science-predict-future-sales/item_categories.csv')

In [3]:
sales_Data

date,date_block_num,shop_id,item_id,item_price,item_cnt_day
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
02.01.2013,0,59,22154,999.00,1
03.01.2013,0,25,2552,899.00,1
05.01.2013,0,25,2552,899.00,-1
06.01.2013,0,25,2554,1709.05,1
15.01.2013,0,25,2555,1099.00,1
10.01.2013,0,25,2564,349.00,1
02.01.2013,0,25,2565,549.00,1
04.01.2013,0,25,2572,239.00,1
11.01.2013,0,25,2572,299.00,1


In [4]:
sales_Data$date<-as.Date(sales_Data$date,format = "%d.%m.%Y")
sales_Data<-sales_Data %>% arrange(sales_Data$date)

In [5]:
train_monthly<-sales_Data%>%inner_join(itemsData,by = "item_id")%>%filter((item_price<50000)&(item_cnt_day < 1000)&(item_price > 0) & (item_cnt_day >= 0))%>%transmute(date,date_block_num,shop_id,item_category_id, item_id, item_price, item_cnt_day,revenue=item_price*item_cnt_day)
train_monthly[train_monthly['shop_id'] == 0, 'shop_id'] = 57
train_monthly[train_monthly['shop_id']== 1, 'shop_id'] = 58
train_monthly[train_monthly['shop_id'] == 11, 'shop_id'] = 10
train_monthly[train_monthly['shop_id'] == 40, 'shop_id'] = 39
train_monthly

date,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day,revenue
<date>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>
2013-01-01,0,19,40,18976,399,1,399
2013-01-01,0,19,55,18284,199,1,199
2013-01-01,0,19,55,18320,199,1,199
2013-01-01,0,19,55,18329,299,1,299
2013-01-01,0,19,62,19367,399,1,399
2013-01-01,0,19,40,18823,299,1,299
2013-01-01,0,19,30,19413,399,1,399
2013-01-01,0,19,62,19366,399,1,399
2013-01-01,0,19,40,18721,149,1,149


In [6]:
train_monthly<-train_monthly%>%transmute(date_block_num,shop_id, item_category_id, item_id,item_cnt_day,item_price,revenue)%>%group_by(date_block_num,shop_id, item_category_id, item_id)%>%summarize(item_cnt_month=sum(item_cnt_day),mean_item_cnt=mean(item_cnt_day),mean_item_price=mean(item_price),revenue_month=sum(revenue))
train_monthly

`summarise()` has grouped output by 'date_block_num', 'shop_id', 'item_category_id'. You can override using the
`.groups` argument.


date_block_num,shop_id,item_category_id,item_id,item_cnt_month,mean_item_cnt,mean_item_price,revenue_month
<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
0,2,2,5572,9,1.285714,1532.857,13910.00
0,2,2,5573,2,1.000000,924.000,1848.00
0,2,2,5575,4,1.000000,955.005,3820.02
0,2,2,5576,3,1.000000,2490.000,7470.00
0,2,2,5632,1,1.000000,2390.000,2390.00
0,2,2,5633,1,1.000000,2298.000,2298.00
0,2,2,5643,1,1.000000,2390.000,2390.00
0,2,2,5649,1,1.000000,2190.000,2190.00
0,2,2,13510,2,1.000000,3790.000,7580.00


In [7]:
testData["date_block_num"] <- 34
testData

ID,shop_id,item_id,date_block_num
<int>,<int>,<int>,<dbl>
0,5,5037,34
1,5,5320,34
2,5,5233,34
3,5,5232,34
4,5,5268,34
5,5,5039,34
6,5,5041,34
7,5,5046,34
8,5,5319,34


In [8]:
shop_ids<-unique(testData[,'shop_id'])
item_ids<-unique(testData[,'item_id'])
months<-0:33
empty_df<-expand.grid(date_block_num=months,shop_id=shop_ids,item_id=item_ids)
empty_df<-rbind(empty_df,testData[,c('date_block_num','shop_id','item_id')])
empty_df

date_block_num,shop_id,item_id
<dbl>,<int>,<int>
0,5,5037
1,5,5037
2,5,5037
3,5,5037
4,5,5037
5,5,5037
6,5,5037
7,5,5037
8,5,5037


In [9]:
train_monthly<-empty_df%>%left_join(train_monthly,by=c('date_block_num','shop_id','item_id'))
train_monthly[is.na(train_monthly)]<-0
train_monthly<-train_monthly[,c('date_block_num','shop_id','item_id','item_cnt_month','mean_item_cnt','mean_item_price','revenue_month')]%>%inner_join(itemsData,by = "item_id")
train_monthly<-train_monthly[,c('date_block_num','shop_id','item_category_id','item_id','item_cnt_month','mean_item_cnt','mean_item_price','revenue_month')]
train_monthly['item_cnt_month']<-lapply(train_monthly['item_cnt_month'],function(x){ifelse(x>20,20,ifelse(x<0,0,x))})
train_monthly['month']<-train_monthly['date_block_num']%%12
train_monthly

date_block_num,shop_id,item_category_id,item_id,item_cnt_month,mean_item_cnt,mean_item_price,revenue_month,month
<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,5,19,5037,0,0,0,0,0
1,5,19,5037,0,0,0,0,1
2,5,19,5037,0,0,0,0,2
3,5,19,5037,0,0,0,0,3
4,5,19,5037,0,0,0,0,4
5,5,19,5037,0,0,0,0,5
6,5,19,5037,0,0,0,0,6
7,5,19,5037,0,0,0,0,7
8,5,19,5037,0,0,0,0,8


In [10]:
date_block_shop_mean<-train_monthly%>%group_by(date_block_num,shop_id)%>%summarize(date_block_shop_mean=mean(item_cnt_month))
date_block_shop_mean
train_monthly<-train_monthly%>%left_join(date_block_shop_mean,by=c('date_block_num', 'shop_id'))

`summarise()` has grouped output by 'date_block_num'. You can override using the `.groups` argument.


date_block_num,shop_id,date_block_shop_mean
<dbl>,<dbl>,<dbl>
0,2,0.04901961
0,3,0.03666667
0,4,0.08901961
0,5,0.00000000
0,6,0.15862745
0,7,0.10882353
0,10,0.03843137
0,12,0.06294118
0,14,0.08607843


In [11]:
date_block_item_mean<-train_monthly%>%group_by(date_block_num,item_id)%>%summarize(date_block_item_mean=mean(item_cnt_month))
date_block_item_mean
train_monthly<-train_monthly%>%left_join(date_block_item_mean,by=c('date_block_num', 'item_id'))
date_category_mean<-train_monthly%>%group_by(date_block_num,item_category_id)%>%summarize(date_cat_mean=mean(item_category_id))
date_category_mean
train_monthly<-train_monthly%>%left_join(date_category_mean,by=c('date_block_num', 'item_category_id'))
train_monthly[is.na(train_monthly)]<-0

`summarise()` has grouped output by 'date_block_num'. You can override using the `.groups` argument.


date_block_num,item_id,date_block_item_mean
<dbl>,<int>,<dbl>
0,30,0.00000000
0,31,0.00000000
0,32,4.50000000
0,33,1.07142857
0,38,0.00000000
0,42,0.04761905
0,45,0.16666667
0,51,0.26190476
0,53,0.11904762


`summarise()` has grouped output by 'date_block_num'. You can override using the `.groups` argument.


date_block_num,item_category_id,date_cat_mean
<dbl>,<int>,<dbl>
0,0,0
0,2,2
0,3,3
0,5,5
0,6,6
0,7,7
0,9,9
0,11,11
0,12,12


In [12]:
temp<- train_monthly[,c("date_block_num", "shop_id","item_id","item_cnt_month","revenue_month","mean_item_cnt","mean_item_price","date_block_shop_mean","date_block_item_mean","date_cat_mean")]
for (i in 1:3){
    shifted<-temp
    names(shifted) <-c("date_block_num", "shop_id", "item_id", sprintf("item_cnt_month_lag_%d",i),sprintf("revenue_month_lag_%d",i),sprintf("mean_item_cnt_lag_%d",i),sprintf("mean_item_price_lag_%d",i),sprintf("date_block_shop_mean_lag_%d",i),sprintf("date_block_item_mean_lag_%d",i),sprintf("date_cat_mean_lag_%d",i))
    shifted[,"date_block_num"]<-shifted[,"date_block_num"]+i
    train_monthly<-train_monthly%>%left_join(shifted,by=c('date_block_num','shop_id','item_id'))
}
train_monthly[is.na(train_monthly)]<-0
train_monthly['qmean_item_cnt']<-apply(train_monthly[,c('item_cnt_month_lag_1', 
                                    'item_cnt_month_lag_2', 
                                    'item_cnt_month_lag_3')], 1, mean, na.rm=TRUE)
train_monthly['qmax_item_cnt'] <- apply(train_monthly[,c('item_cnt_month_lag_1', 
                                    'item_cnt_month_lag_2', 
                                    'item_cnt_month_lag_3')], 1, max, na.rm=TRUE)
train_monthly['qstd_item_cnt'] <- apply(train_monthly[,c('item_cnt_month_lag_1', 
                                    'item_cnt_month_lag_2', 
                                    'item_cnt_month_lag_3')], 1, sd, na.rm=TRUE)
train_monthly['qmin_item_cnt'] <- apply(train_monthly[,c('item_cnt_month_lag_1', 
                                    'item_cnt_month_lag_2', 
                                    'item_cnt_month_lag_3')], 1, min, na.rm=TRUE)
train_monthly

date_block_num,shop_id,item_category_id,item_id,item_cnt_month,mean_item_cnt,mean_item_price,revenue_month,month,date_block_shop_mean,⋯,revenue_month_lag_3,mean_item_cnt_lag_3,mean_item_price_lag_3,date_block_shop_mean_lag_3,date_block_item_mean_lag_3,date_cat_mean_lag_3,qmean_item_cnt,qmax_item_cnt,qstd_item_cnt,qmin_item_cnt
<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,5,19,5037,0,0,0,0,0,0.00000000,⋯,0,0,0,0.00000000,0.0000000,0,0.0000000,0,0.0000000,0
1,5,19,5037,0,0,0,0,1,0.03666667,⋯,0,0,0,0.00000000,0.0000000,0,0.0000000,0,0.0000000,0
2,5,19,5037,0,0,0,0,2,0.07098039,⋯,0,0,0,0.00000000,0.0000000,0,0.0000000,0,0.0000000,0
3,5,19,5037,0,0,0,0,3,0.04941176,⋯,0,0,0,0.00000000,0.0000000,19,0.0000000,0,0.0000000,0
4,5,19,5037,0,0,0,0,4,0.05823529,⋯,0,0,0,0.03666667,0.0000000,19,0.0000000,0,0.0000000,0
5,5,19,5037,0,0,0,0,5,0.06686275,⋯,0,0,0,0.07098039,0.0000000,19,0.0000000,0,0.0000000,0
6,5,19,5037,0,0,0,0,6,0.06274510,⋯,0,0,0,0.04941176,0.0000000,19,0.0000000,0,0.0000000,0
7,5,19,5037,0,0,0,0,7,0.08450980,⋯,0,0,0,0.05823529,0.0000000,19,0.0000000,0,0.0000000,0
8,5,19,5037,0,0,0,0,8,0.08039216,⋯,0,0,0,0.06686275,0.0000000,19,0.0000000,0,0.0000000,0


In [13]:
minMax <- function(x) {
  (x - min(x)) / (max(x) - min(x))
}
y_min<-min(train_monthly$item_cnt_month)
y_max<-max(train_monthly$item_cnt_month)
train_monthly[,c('mean_item_cnt','mean_item_price','revenue_month','date_block_shop_mean','date_block_item_mean','date_block_item_mean_lag_1','date_block_item_mean_lag_2','date_block_item_mean_lag_3','date_block_shop_mean_lag_1','date_block_shop_mean_lag_2','date_block_shop_mean_lag_3','item_cnt_month_lag_1','revenue_month_lag_1','mean_item_cnt_lag_1','mean_item_price_lag_1','item_cnt_month_lag_2','revenue_month_lag_2','mean_item_cnt_lag_2','mean_item_price_lag_2','item_cnt_month_lag_3','revenue_month_lag_3','mean_item_cnt_lag_3','mean_item_price_lag_3','date_cat_mean','date_cat_mean_lag_1','date_cat_mean_lag_2','date_cat_mean_lag_3')]<-apply(train_monthly[,c('mean_item_cnt','mean_item_price','revenue_month','date_block_shop_mean','date_block_item_mean','date_block_item_mean_lag_1','date_block_item_mean_lag_2','date_block_item_mean_lag_3','date_block_shop_mean_lag_1','date_block_shop_mean_lag_2','date_block_shop_mean_lag_3','item_cnt_month_lag_1','revenue_month_lag_1','mean_item_cnt_lag_1','mean_item_price_lag_1','item_cnt_month_lag_2','revenue_month_lag_2','mean_item_cnt_lag_2','mean_item_price_lag_2','item_cnt_month_lag_3','revenue_month_lag_3','mean_item_cnt_lag_3','mean_item_price_lag_3','date_cat_mean','date_cat_mean_lag_1','date_cat_mean_lag_2','date_cat_mean_lag_3')],2,minMax)

In [14]:
clip<-function(x){
    ifelse(x>20,20,ifelse(x<0,0,x))
}

In [15]:
train<-train_monthly[(train_monthly$date_block_num<33&train_monthly$date_block_num>2),-which(names(train_monthly) %in% c('mean_item_price','revenue_month','mean_item_cnt', 'date_block_shop_mean','date_block_item_mean','date_cat_mean'))]
valid<-train_monthly[train_monthly$date_block_num==33,-which(names(train_monthly) %in% c('mean_item_price','revenue_month','mean_item_cnt','date_block_shop_mean','date_block_item_mean','date_cat_mean'))]
test<-train_monthly[train_monthly$date_block_num==34,-which(names(train_monthly) %in% c('item_cnt_month','mean_item_price','revenue_month','mean_item_cnt','date_block_shop_mean','date_block_item_mean','date_cat_mean'))]
tem<-glm(item_cnt_month~.,data=train, na.action = na.exclude)#glm(item_cnt_month~.,data=train)
summary(tem)
sqrt(mean((clip(predict(tem, train))-train[,'item_cnt_month'])^2))
sqrt(mean((clip(predict(tem, valid))-valid[,'item_cnt_month'])^2))


Call:
glm(formula = item_cnt_month ~ ., data = train, na.action = na.exclude)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-19.6972   -0.0977   -0.0449   -0.0032   20.0572  

Coefficients: (4 not defined because of singularities)
                             Estimate Std. Error  t value Pr(>|t|)    
(Intercept)                -8.544e-03  1.419e-03   -6.021 1.73e-09 ***
date_block_num             -7.049e-04  3.812e-05  -18.491  < 2e-16 ***
shop_id                     1.966e-05  1.752e-05    1.122    0.262    
item_category_id           -1.083e-03  1.924e-05  -56.307  < 2e-16 ***
item_id                    -1.198e-06  5.113e-08  -23.428  < 2e-16 ***
month                       1.172e-02  9.802e-05  119.612  < 2e-16 ***
item_cnt_month_lag_1        6.782e+00  2.140e-02  316.966  < 2e-16 ***
revenue_month_lag_1         5.835e+00  3.941e-01   14.808  < 2e-16 ***
mean_item_cnt_lag_1         9.661e+00  4.769e-01   20.258  < 2e-16 ***
mean_item_price_lag_1      -2.9

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"


[1] 0.7792925

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"


[1] 0.9007641

In [16]:
tem_2<-glm(item_cnt_month~date_block_num+item_category_id+item_id+month+item_cnt_month_lag_1+revenue_month_lag_1+mean_item_cnt_lag_1+mean_item_price_lag_1+date_block_shop_mean_lag_1+date_block_item_mean_lag_1+item_cnt_month_lag_2+revenue_month_lag_2+mean_item_cnt_lag_2+date_block_shop_mean_lag_2+date_block_item_mean_lag_2+item_cnt_month_lag_3+revenue_month_lag_3+mean_item_cnt_lag_3+mean_item_price_lag_3+date_block_shop_mean_lag_3+date_block_item_mean_lag_3+qmax_item_cnt+qstd_item_cnt+qmin_item_cnt,data=train, na.action = na.exclude)
summary(tem_2)
sqrt(mean((clip(predict(tem_2, train))-train[,'item_cnt_month'])^2))
sqrt(mean((clip(predict(tem_2, valid))-valid[,'item_cnt_month'])^2))


Call:
glm(formula = item_cnt_month ~ date_block_num + item_category_id + 
    item_id + month + item_cnt_month_lag_1 + revenue_month_lag_1 + 
    mean_item_cnt_lag_1 + mean_item_price_lag_1 + date_block_shop_mean_lag_1 + 
    date_block_item_mean_lag_1 + item_cnt_month_lag_2 + revenue_month_lag_2 + 
    mean_item_cnt_lag_2 + date_block_shop_mean_lag_2 + date_block_item_mean_lag_2 + 
    item_cnt_month_lag_3 + revenue_month_lag_3 + mean_item_cnt_lag_3 + 
    mean_item_price_lag_3 + date_block_shop_mean_lag_3 + date_block_item_mean_lag_3 + 
    qmax_item_cnt + qstd_item_cnt + qmin_item_cnt, data = train, 
    na.action = na.exclude)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-19.5357   -0.0977   -0.0449   -0.0033   20.0577  

Coefficients:
                             Estimate Std. Error  t value Pr(>|t|)    
(Intercept)                -7.807e-03  1.310e-03   -5.959 2.53e-09 ***
date_block_num             -7.025e-04  3.812e-05  -18.431  < 2e-16 ***
item_cat

[1] 0.7792975

[1] 0.9007872

In [17]:
test_y<-predict(tem_2, test)#(predict(tem_2, test)*(y_max-y_min))+y_min
test_y<-clip(test_y)#ifelse(test_y>20,20,ifelse(test_y<0,0,test_y))

In [18]:
results <- data.frame (ID  = testData$ID,
                  item_cnt_month = test_y
                  )

In [19]:
write.csv(results,file="results/data.csv",row.names = FALSE)
rm(list=c('tem','tem_2','test_y','results'))

In [20]:
library(glmnet)
x_train <- model.matrix(item_cnt_month~., train)[,-1]
y_train<-train$item_cnt_month
x_valid <- model.matrix(item_cnt_month~., valid)[,-1]
y_valid<-valid$item_cnt_month
x_test <- data.matrix(test)
ridge_mod_cv = cv.glmnet(x_train, y_train,nfolds=6, alpha = 0)
fit <- glmnet(x_train, y_train, alpha = 0)
sqrt(mean((clip(predict(fit, s = ridge_mod_cv$lambda.min, newx = x_train))-train[,'item_cnt_month'])^2))
sqrt(mean((clip(predict(fit, s = ridge_mod_cv$lambda.min, newx = x_valid))-valid[,'item_cnt_month'])^2))
test_y<-predict(fit,s = ridge_mod_cv$lambda.min,newx=x_test)
test_y<-clip(test_y)
results <- data.frame (ID  = testData$ID,
                  item_cnt_month = test_y
                  )
colnames(results)<-c('ID','item_cnt_month')
write.csv(results,file="results/data_1.csv",row.names = FALSE)
rm(list=c('x_train','y_train','x_valid','y_valid','x_test','fit','results','ridge_mod_cv'))

Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack


Loaded glmnet 4.1-3



[1] 0.7801895

[1] 0.8987865

In [21]:
# library(rpart)

In [22]:
# # gc()
# fit <- rpart(item_cnt_month~., 
#              method = "anova",maxdepth=9,data = train)

In [23]:
# sqrt(mean((clip(predict(fit,train,method="anova"))-train[,'item_cnt_month'])^2))
# sqrt(mean((clip(predict(fit,valid,method="anova"))-valid[,'item_cnt_month'])^2))
# test_y<-predict(fit, test,method="anova")#(predict(tem_2, test)*(y_max-y_min))+y_min
# test_y<-clip(test_y)#ifelse(test_y>20,20,ifelse(test_y<0,0,test_y))
# test_y
# results_1 <- data.frame (ID  = testData$ID,
#                   item_cnt_month = test_y
#                   )
# results_1

In [24]:
# write.csv(results_1,file="results/data_2.csv",row.names = FALSE)